In [ ]:
# Установка библиотек (в случае, если ранее они не были установлены)
!pip install bs4 pandas requests selenium

In [34]:
import time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# путь к вебдрайверу
# PATH = b"C:\Users\zzbin\Desktop\Google Chrome\chromedriver.exe"
PATH = input() 
service = Service(PATH)
driver = webdriver.Chrome(service=service)


In [35]:
account_name = input('Введите название интересуещего аккаунта')
# driver.get('https://www.tiktok.com/@chef_kostya')

# проверка на существование аккаунта
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.141 YaBrowser/22.3.2.644'}
resp = requests.get('https://www.tiktok.com/@' + account_name, headers=headers)
while not resp.ok:
    print('Такого аккаунта не существует либо он был введен неверно.\nПопробуйте еще раз')
    account_name = input()
    resp = requests.get('https://www.tiktok.com/@' + account_name, headers=headers)
    

# открываем сайт 
driver.get('https://www.tiktok.com/@' + account_name)

last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(18):
    # Скроллим вниз по странице
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Ждем, пока страницы прогрузится полностью
    time.sleep(1.5)

    # Считает новую высоту скроллинга (положение страницы) со старой
    new_height = driver.execute_script("return document.body.scrollHeight")
    # если это конец страницы, перестает скроллить 
    if new_height == last_height:
        break
    last_height = new_height
    
html = driver.page_source
driver.quit()

Такого аккаунта не существует либо он был введен неверно.
Попробуйте еще раз


In [ ]:
soup = BeautifulSoup(html, 'lxml')
links_list = list(map(lambda x: x.get('href'), soup.find_all('a', {'href': lambda x: x.startswith('https://www.tiktok.com/@')})))

def decode_number(number):
    if 'K' in number:
        return int(float(number.replace('K', '')) * 1_000)
    elif 'M' in number:
        return int(float(number.replace('M', '')) * 1_000_000)
    return int(number)
video_views =  list(map(lambda x: decode_number(x.text), soup.find_all('strong', {'data-e2e': 'video-views'})))

def get_all_info(links):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.141 YaBrowser/22.3.2.644'}
    dates_list, likes_list, comments_list, reposts_list, videos_text_list = [], [], [], [], []
    
    def get_info_by_link(url):
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.text, 'lxml')
        
        likes = soup.find('strong', {'data-e2e':"like-count"}).text
        comments = soup.find('strong', {'data-e2e':"comment-count"}).text
        reposts = soup.find('strong', {'data-e2e':"share-count"}).text
        
        date = soup.find('a', class_='tiktok-1wozk2e-StyledAuthorAnchor e1aily491')
        date = '.'.join(re.findall(r'\d+', date.text)) 
        video_text = re.findall(r'(.+)TikTokUpload', soup.text)[0].strip()
        return date, likes, comments, reposts, video_text
    
    for link in links:      
        info = get_info_by_link(link)
        dates_list.append(info[0])
        likes_list.append(info[1])
        comments_list.append(info[2])
        reposts_list.append(info[3])
        videos_text_list.append(info[4])
        
    likes_list = list(map(lambda x: decode_number(x), likes_list))
    comments_list = list(map(lambda x: decode_number(x), comments_list))
    reposts_list = list(map(lambda x: decode_number(x), reposts_list))
    return dates_list, likes_list, comments_list, reposts_list, videos_text_list

dates_list, likes_list, comments_list, reposts_list, videos_text_list = get_all_info(links_list)
info = {'link':links_list, 'views': video_views, 'date': dates_list, 'comments': comments_list, 'reposts': reposts_list, 'video_text': videos_text_list}

df = pd.DataFrame(info)
df['date'] = df['date'].apply(lambda x: time.ctime().split()[-1] + '.' + x if not re.match(r'\d{4}\.\d{1,2}\.\d{1,2}', x) else x)


In [ ]:
df

In [ ]:
filepath = input()
filename = input()
df.to_excel(f"{filepath + filename}.xlsx") 